# <a id="top">Lab 2: Response Level Detection</a>

## Detecting LLM Hallucinations Through Response Consistency Analysis

For many applications built on closed-source, proprietary models accessed via APIs, we don't have access to the model's internal states (like weights or logits) or ground truth data to compare against. **Response Level Detection** techniques are designed for this scenario, treating the model as an opaque system and relying only on its input-output behavior to assess reliability and detect potential hallucinations.

In this notebook, we will implement and compare three cutting-edge response level detection methods that work without requiring reference answers or model internals. These methods analyze the **consistency and coherence of multiple model responses** to the same prompt.

We'll implement and compare three cutting-edge detection methods:
1. **Semantic Similarity Analysis** - Measuring consistency across multiple model responses using embeddings
2. **Non-Contradiction Probability** - Using Natural Language Inference (NLI) to detect logical contradictions
3. **Normalized Semantic Negentropy (NSN)** - Measuring semantic uncertainty through response clustering

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>💡 Key Learning Objectives</h4>
    <ul>
        <li>Understand Response Level hallucination detection techniques</li>
        <li>Learn how to implement semantic similarity analysis with sentence embeddings</li>
        <li>Master contradiction detection using Natural Language Inference (NLI) models</li>
        <li>Apply semantic entropy methods for uncertainty quantification</li>
        <li>Build AI agents with Strands Agents SDK that automatically detect hallucinations</li>
    </ul>
</div>
<br/>

## Use-Case Overview

Response Level hallucination detection is crucial when:
- You don't have ground truth answers to compare against
- You need to assess model reliability in production without access to model internals
- You want to identify when a model is "making things up"
- You need to implement confidence scoring for AI responses

Our approach leverages the principle that **reliable knowledge should be consistent** across multiple generations with different sampling parameters, while hallucinations tend to vary significantly.


## Sections

This notebook has the following sections:

1. [Environment Setup and Model Configuration](#1.-Environment-Setup-and-Model-Configuration)
2. [Method 1: Semantic Similarity Detection](#2.-Method-1:-Semantic-Similarity-Detection)
3. [Method 2: Non-Contradiction Probability](#3.-Method-2:-Non-Contradiction-Probability)
4. [Method 3: Normalized Semantic Negentropy (NSN)](#4.-Method-3:-Normalized-Semantic-Negentropy-(NSN))
5. [Comparative Analysis Across Models](#5.-Comparative-Analysis-Across-Models)
6. [Building Production-Ready Agents with Semantic Similarity](#6.-Building-Production-Ready-Agents-with-Semantic-Similarity)
7. [Conclusion and Best Practices](#7.-Conclusion-and-Best-Practices)
8. [(Optional) Challenge Exercises](#8.-(Optional)-Challenge-Exercises)
    
Please work from top to bottom and don't skip sections as this could lead to error messages due to missing dependencies.

----

## 1. Environment Setup and Model Configuration
(<a href="#top">Go to top</a>)

**If you haven't already** installed the workshop's dependencies (from [pyproject.toml](./pyproject.toml)), you can un-comment (remove `# `) and run the below cell to do so. We've commented it out by default, assuming you already ran it at the start of lab 0:

In [ ]:
# %pip install -q -e .

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; padding-top: 15px;">
    <h4>🔄 Restart the kernel after installing</h4>
    <p>
        <strong>IF</strong> you ran the above install command cell, you'll need to restart the
        notebook kernel afterwards for the installations to take full effect.
    </p>
    <p>
        Note that you may see some error notices about dependency conflicts in SageMaker Studio
        environments, but this is okay as long as the installations are completed.
    </p>
</div>
<br/>

With the installation complete, you're ready to import the libraries we'll use in the notebook:

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
from dataclasses import dataclass
import json
from logging import getLogger, basicConfig
import time
from typing import Callable, Dict, List, Tuple
import warnings

# External Dependencies:
import boto3
import numpy as np
import pandas as pd
from transformers import pipeline, Pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

# Local Utilities:
from hallucination_utils.llm_wrappers import BedrockLLM, SageMakerVLLM

warnings.filterwarnings("ignore")
logger = getLogger()

print("All packages imported successfully!")

### Configure Foundation Models

We'll test the hallucination detection models in this lab on *both* the Amazon Bedrock Foundation Model as used in lab 1, and the Open Weights model we deployed on Amazon SageMaker AI in lab 0.

Although the underlying APIs for these two services are different (see Bedrock [`converse()`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html) and SageMaker [`invoke_endpoint()`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime/client/invoke_endpoint.html) in Python), all we really need for this lab is to be able to send in a text prompt and get back the text response. Open Source libraries like [LangChain](https://docs.langchain.com/oss/python/langchain/models) and [LiteLLM](https://docs.litellm.ai/docs/providers) offer standardised wrappers for a wide variety of LLM hosting providers, but it's also pretty easy to build your own if you value minimizing dependencies - which is what we've done here. If you're curious, you can see the implementations in [hallucination_utils/llm_wrappers.py](hallucination_utils/llm_wrappers.py)

#### Amazon Bedrock

First, let's connect to Anthropic Claude on Amazon Bedrock as we used earlier in lab 1:

In [ ]:
BEDROCK_MODEL_ID = "global.anthropic.claude-haiku-4-5-20251001-v1:0"

claude_llm = BedrockLLM(model_id=BEDROCK_MODEL_ID)
print(f"Using Bedrock model '{claude_llm.model_id}'")

test_prompt = "What is 2 + 2?"
print(f"\nQuestion: {test_prompt}\nAnswer:")
print(claude_llm.invoke(test_prompt))

#### Amazon SageMaker AI

Next, we'll connect to the Open Weight GPT-OSS model you deployed to Amazon SageMaker AI in lab 0.

For this, we'll need the endpoint name and inference component name. The below cell will try to retrieve those automatically from where they were `%store`d in the lab 0 notebook:

In [ ]:
%store -r endpoint_name
%store -r inference_component_name
# (Alternatively if this is not working, could just define these variables here directly)

try:
    endpoint_name, inference_component_name
except NameError as e:
    raise RuntimeError(
        "SageMaker endpoint not found. Please check you've run lab 0 first!"
    )

In [ ]:
gpt_oss_llm = SageMakerVLLM(
    endpoint_name=endpoint_name,
    inference_component_name=inference_component_name,
)
print(
    "Using SageMaker endpoint '%s'\n  └─ inf component '%s'"
    % (gpt_oss_llm.endpoint_name, gpt_oss_llm.inference_component_name)
)

test_prompt = "What is 2 + 2?"
print(f"\nQuestion: {test_prompt}\nAnswer:")
print(gpt_oss_llm.invoke(test_prompt))

### Generating Multiple Responses

Since all the methods we'll explore here depend on generating and comparing **multiple** responses to a single input prompt, we'll set up a re-usable utility to handle that (and ignore if a small fraction of the generations fail due to transient errors):

In [ ]:
def generate_responses(
    models: list, prompt: str, min_gen_pct: float = 0.8
) -> list[str]:
    n_models = len(models)
    print(f'💬 Generating {n_models} responses for prompt:\n"{prompt}"')
    results = []
    for ix, model in enumerate(models):
        try:
            res = model.invoke(prompt)
            results.append(res)
            print(f'🤖 Model {ix + 1}/{n_models} responded:\n"{res}"')
        except Exception as e:
            # In case of intermittent throttling or other issues during the workshop,
            # we'll try to carry on if some of the responses fail to generate
            logger.info("Failed to generate model response: %s", e)
    n_gens = len(results)
    n_fails = n_models - n_gens
    if n_fails:
        gen_pct = n_fails / n_models
        msg = (
            "%s of %s models (%.1%) failed to generate a response - trying to continue"
            % (n_fails, n_models, gen_pct)
        )
        if gen_pct < min_gen_pct:
            raise RuntimeError(msg)
        else:
            logger.warning(msg)
    print()
    return results

The actual approach you take to generate your multiple responses is flexible, depending on the analysis you're performing. For example, it could be:
- Multiple calls to the exact same model with the exact same inference parameters (Since LLMs are typically non-deterministic)
- Calls to the same model with a range of inference parameters (like varying the temperature or maximum length a little around your target value)
- Calls to multiple different models

In this lab we'll mainly use one model at a time, but vary the temperature to encourage some diversity in the responses. Run the cells below to set up these groups for both Claude on Bedrock and your Open Weight model on SageMaker:

In [ ]:
n_samples = 5
claude_models = [
    BedrockLLM(
        model_id=BEDROCK_MODEL_ID, temperature=0.1 + 0.9 * (ix / (n_samples - 1))
    )
    for ix in range(n_samples)
]

answers = generate_responses(
    claude_models, "Choose a preferred name for yourself, and reply with only the name."
)

In [ ]:
n_samples = 5
gpt_oss_models = [
    SageMakerVLLM(
        endpoint_name=endpoint_name,
        inference_component_name=inference_component_name,
        temperature=0.1 + 0.9 * (ix / (n_samples - 1)),
    )
    for ix in range(n_samples)
]

answers = generate_responses(
    gpt_oss_models,
    "Choose a preferred name for yourself, and reply with only the name.",
)

Now we're comfortable with generating a batch of possible answers to a given prompt, let's dive in to the methods for analyzing them for lack of confidence and likely hallucinations.

## 2. Method 1: Semantic Similarity Detection
(<a href="#top">Go to top</a>)

### How It Works

Semantic Similarity Detection works on the principle that **reliable information should be expressed consistently**, even when generated multiple times with different parameters.

**The Process:**

Given multiple candidate responses (in this case we'll use the same model, but vary the temperature):
1. **Create embeddings**: Convert each response to a high-dimensional vector representation
2. **Calculate pairwise similarities**: Measure cosine similarity between all response pairs
3. **Assess consistency**: High similarity = reliable, low similarity = potential hallucination

**Key Principle**: Reliable information should be expressed consistently, regardless of sampling parameters.

In [ ]:
def semantic_similarity_detection(
    responses: list[str],
    embedding_model=SentenceTransformer("all-MiniLM-L6-v2"),
) -> dict:
    """Analyze semantic similarity across multiple responses to detect hallucinations

    Args:
        responses: The multiple alternative responses generated by the model
        embedding_model: A SentenceTransformer-compatible model to calculate text embedding vectors

    Returns:
        Dictionary with similarity metrics and confidence assessment
    """
    n_responses = len(responses)
    if n_responses < 2:
        raise ValueError(
            f"Need at least 2 responses for semantic similarity comparison. Got {n_responses}"
        )
    print(f"🔍 Analyzing semantic similarity of {n_responses} candidate responses...")

    # Step 1: Create a semantic embedding vector for each response
    print(f"├─ Creating semantic embeddings...")
    embeddings = embedding_model.encode(responses)

    # Step 2: Calculate pairwise similarities
    print(f"├─ Calculating semantic similarities...")
    similarity_matrix = cosine_similarity(embeddings)

    # Get all pairwise similarities (excluding diagonal similarity-with-self)
    mask = np.ones(similarity_matrix.shape, dtype=bool)
    np.fill_diagonal(mask, False)
    similarities = similarity_matrix[mask]

    # Step 3: Calculate metrics and determine confidence level
    mean_similarity = np.mean(similarities)
    std_similarity = np.std(similarities)
    min_similarity = np.min(similarities)
    max_similarity = np.max(similarities)

    if mean_similarity > 0.95 and std_similarity < 0.03:
        confidence = "🟢 HIGH"
        interpretation = "Responses are highly consistent - reliable knowledge"
    elif mean_similarity > 0.85 and std_similarity < 0.08:
        confidence = "🟡 MEDIUM"
        interpretation = "Moderate consistency - some uncertainty present"
    else:
        confidence = "🔴 LOW"
        interpretation = "Inconsistent responses - possible hallucination"

    # Display results
    print(f"├─ METRICS:")
    print(f"│   ├─ Mean Similarity:  {mean_similarity:.3f}")
    print(f"│   ├─ Std Deviation:    {std_similarity:.3f}")
    print(f"│   ├─ Min Similarity:   {min_similarity:.3f}")
    print(f"│   └─ Max Similarity:   {max_similarity:.3f}")
    print(f"└─ CONFIDENCE: {confidence}")
    print(f"    └─ {interpretation}")

    return {
        "method": "Semantic Similarity",
        "num_responses": len(responses),
        "mean_similarity": round(mean_similarity, 3),
        "std_similarity": round(std_similarity, 3),
        "min_similarity": round(min_similarity, 3),
        "confidence": confidence,
        "interpretation": interpretation,
        "responses": responses,
    }


# Don't worry about the red warnings that might get generated here:
print("✅ Set up semantic_similarity_detection")

### Test Semantic Similarity Detection

Let's test this method with a question that should have a clear, factual answer.

We'll compare 5 responses generated using Claude on Bedrock, at a range of different temperatures:

In [ ]:
semsim_bedrock_responses = generate_responses(
    claude_models, "What album was Dua Lipa's 'Hallucinate' featured on?"
)

semantic_similarity_detection(semsim_bedrock_responses)

### Optional: Test with SageMaker Endpoint

If you have SageMaker endpoints with inference components available, you can test them here. Otherwise, skip to the next section.

In [ ]:
semsim_sm_responses = generate_responses(
    gpt_oss_models,
    "What album was Dua Lipa's 'Hallucinate' featured on?",
)

semantic_similarity_detection(semsim_sm_responses)

## 3. Method 2: Non-Contradiction Probability
(<a href="#top">Go to top</a>)

### How It Works

Non-Contradiction Probability uses [**Natural Language Inference (NLI)**](https://huggingface.co/tasks/text-classification#natural-language-inference-nli) to detect logical inconsistencies between multiple model responses.

**The Process:**

Given multiple candidate responses (again in this case we'll use the same model, but vary the temperature):
1. **Pairwise contradiction detection**: Compare every response pair using an NLI model to identify logical contradictions
2. **Calculate frequency metrics**: The overall frequency of contradictions and the variance between models
3. **Assess confidence**: Determine confidence in the answer based on the rate of contradictions observed

**Key Principle**: Reliable knowledge should not contradict itself, while hallucinations often contain conflicting information.

In [ ]:
def non_contradiction_detection(
    responses: list[str],
    nli_model: Pipeline = pipeline(
        "text-classification",
        model="microsoft/deberta-base-mnli",
        return_all_scores=True,
    ),
) -> dict:
    """Detect hallucinations by analyzing logical contradictions between responses

    Args:
        responses: The multiple alternative responses generated by the model
        nli_model: HF Transformers Pipeline-compatible model for Natural Language Inference

    Returns:
        Dictionary with contradiction analysis metrics and confidence assessment
    """
    n_responses = len(responses)
    if n_responses < 2:
        raise ValueError(f"Need at least 2 responses for comparison. Got {n_responses}")
    print(
        f"🔍 Analyzing non-contradiction probability of {n_responses} candidate responses..."
    )

    # Step 1: Analyze all response pairs for contradictions
    print(f"├─ Analyzing {len(responses)} responses for logical contradictions...")
    non_contradiction_scores = []
    pair_analyses = []
    for i in range(len(responses)):
        for j in range(i + 1, len(responses)):
            try:
                # Check both directions (A→B and B→A) for mutual contradiction
                result1 = nli_model(f"{responses[i]} [SEP] {responses[j]}")
                result2 = nli_model(f"{responses[j]} [SEP] {responses[i]}")

                # Extract contradiction probabilities
                contra_prob1 = 0.0
                contra_prob2 = 0.0

                # Parse NLI results
                if isinstance(result1, list) and len(result1) > 0:
                    inner_list1 = (
                        result1[0] if isinstance(result1[0], list) else result1
                    )
                    for item in inner_list1:
                        if (
                            isinstance(item, dict)
                            and item.get("label") == "CONTRADICTION"
                        ):
                            contra_prob1 = item.get("score", 0.0)
                            break

                if isinstance(result2, list) and len(result2) > 0:
                    inner_list2 = (
                        result2[0] if isinstance(result2[0], list) else result2
                    )
                    for item in inner_list2:
                        if (
                            isinstance(item, dict)
                            and item.get("label") == "CONTRADICTION"
                        ):
                            contra_prob2 = item.get("score", 0.0)
                            break

                # Calculate average contradiction probability
                avg_contradiction = (contra_prob1 + contra_prob2) / 2
                non_contradiction_score = 1 - avg_contradiction

                non_contradiction_scores.append(non_contradiction_score)
                pair_analyses.append(
                    {
                        "pair": f"{i + 1}-{j + 1}",
                        "contradiction": avg_contradiction,
                        "non_contradiction": non_contradiction_score,
                    }
                )

                print(
                    "│   ├─ Pair {}-{}: Contradiction={:.3f}, Non-contradiction={:.3f}".format(
                        i + 1, j + 1, avg_contradiction, non_contradiction_score
                    )
                )

            except Exception as e:
                logger.warning(f"Error analyzing pair {i + 1}-{j + 1}: {str(e)}")
                non_contradiction_scores.append(0.5)  # Neutral score for errors

    if not non_contradiction_scores:
        raise RuntimeError("Failed to analyze *any* response pairs with NLI")

    # Step 2: Calculate overall metrics
    mean_ncp = np.mean(non_contradiction_scores)
    std_ncp = np.std(non_contradiction_scores)
    min_ncp = np.min(non_contradiction_scores)

    # Step 3: Determine confidence level
    if mean_ncp > 0.8 and min_ncp > 0.7:
        confidence = "🟢 HIGH"
        interpretation = "No significant contradictions - facts are consistent"
    elif mean_ncp > 0.65 and min_ncp > 0.5:
        confidence = "🟡 MEDIUM"
        interpretation = "Minor contradictions detected - mostly consistent"
    else:
        confidence = "🔴 LOW"
        interpretation = "Significant contradictions detected - likely hallucination"

    # Display results
    print(f"├─ METRICS:")
    print(f"│   ├─ Mean Non-Contradiction:  {mean_ncp:.3f}")
    print(f"│   ├─ Std Deviation:           {std_ncp:.3f}")
    print(f"│   └─ Min Non-Contradiction:   {std_ncp:.3f}")
    print(f"└─ CONFIDENCE: {confidence}")
    print(f"    └─ {interpretation}")

    return {
        "method": "Non-Contradiction Probability",
        "num_responses": n_responses,
        "mean_ncp": mean_ncp,
        "std_ncp": std_ncp,
        "min_ncp": min_ncp,
        "pair_analyses": pair_analyses,
        "confidence": confidence,
        "interpretation": interpretation,
        "responses": responses,
    }


# Don't worry about the red warnings that usually get generated here:
print("✅ Set up non_contradiction_detection")

### Test Non-Contradiction Detection

Let's test this method with the same question to see how it compares:

> ℹ️ Note that you *could* re-use the `semsmim_bedrock_responses` from earlier directly, but we've chosen to generate new responses to keep things a bit more interactive.

In [ ]:
ncp_bedrock_responses = generate_responses(
    claude_models, "What album was Dua Lipa's 'Hallucinate' featured on?"
)

non_contradiction_detection(ncp_bedrock_responses)

### Optional: Test with SageMaker Endpoint

Again if you've deployed the SageMaker endpoint(s) in lab 0, you can test them here. Otherwise, skip to the next section:

In [ ]:
ncp_sm_responses = generate_responses(
    gpt_oss_models,
    "What album was Dua Lipa's 'Hallucinate' featured on?",
)

non_contradiction_detection(ncp_sm_responses)

## 4. Method 3: Normalized Semantic Negentropy (NSN)
(<a href="#top">Go to top</a>)

### How It Works

Normalized Semantic Negentropy is the most sophisticated method, measuring **semantic uncertainty** through response clustering.

**The Process:**

Given multiple candidate responses (again in this case we'll use the same model, but vary the temperature):

1. **Semantic Clustering**: Group responses that convey the same meaning (using Natural Language Inference model, as we did earlier)
2. **Entropy Calculation**: Measure how "scattered" the responses are across clusters
3. **Confidence Scoring**: Convert entropy to a confidence score (low entropy = high confidence)

**Key Principle**: Confident knowledge clusters tightly, while uncertain knowledge fragments across many different meanings.

In [ ]:
def semantic_entropy_detection(
    responses: list[str],
    nli_model: Pipeline = pipeline(
        "text-classification",
        model="microsoft/deberta-base-mnli",
        return_all_scores=True,
    ),
) -> dict:
    """Detect hallucinations using Normalized Semantic Negentropy (NSN)

    This method clusters responses by meaning, then calculates entropy of the distribution between
    those clusters to model uncertainty. Lower entropy indicates higher confidence.

    Args:
        responses: The multiple alternative responses generated by the model
        nli_model: HF Transformers Pipeline-compatible model for Natural Language Inference

    Returns:
        Dictionary with entropy metrics and confidence assessment
    """
    n_responses = len(responses)
    if n_responses < 3:
        raise ValueError(
            f"Need at least 3 responses for clustering analysis. Got {n_responses}"
        )
    print(
        f"🔍 Analyzing normalized semantic negentropy of {n_responses} candidate responses..."
    )

    # Step 1: Cluster responses by semantic meaning
    print(f"├─ Clustering responses by semantic meaning...")
    clusters = []

    for i, response in enumerate(responses):
        assigned_to_cluster = False

        # Try to assign to existing cluster
        for cluster in clusters:
            # Compare with representative response from cluster
            representative = responses[cluster[0]]

            # Check for bidirectional entailment (same meaning)
            result1 = nli_model(f"{response} [SEP] {representative}")
            result2 = nli_model(f"{representative} [SEP] {response}")

            # Extract entailment scores
            entail1 = 0.0
            entail2 = 0.0

            if isinstance(result1, list) and len(result1) > 0:
                inner_list1 = result1[0] if isinstance(result1[0], list) else result1
                for item in inner_list1:
                    if isinstance(item, dict) and item.get("label") == "ENTAILMENT":
                        entail1 = item.get("score", 0.0)
                        break

            if isinstance(result2, list) and len(result2) > 0:
                inner_list2 = result2[0] if isinstance(result2[0], list) else result2
                for item in inner_list2:
                    if isinstance(item, dict) and item.get("label") == "ENTAILMENT":
                        entail2 = item.get("score", 0.0)
                        break

            # Use geometric mean for mutual entailment
            mutual_entailment = np.sqrt(entail1 * entail2)

            if mutual_entailment > 0.5:  # Threshold for "same meaning"
                cluster.append(i)
                assigned_to_cluster = True
                break

        # Create new cluster if no match found
        if not assigned_to_cluster:
            clusters.append([i])

    # Step 4: Display cluster information
    print(f"Found {len(clusters)} semantic clusters:")
    for i, cluster in enumerate(clusters):
        print(
            "│   ├─ Cluster {}: Responses {} ({} responses)".format(
                i + 1, [j + 1 for j in cluster], len(cluster)
            )
        )

    # Step 5: Calculate semantic entropy
    cluster_probabilities = [len(cluster) / len(responses) for cluster in clusters]
    semantic_entropy = -sum(p * np.log(p) for p in cluster_probabilities if p > 0)

    # Normalize entropy (0 = all same meaning, 1 = all different meanings)
    max_entropy = np.log(len(responses))
    normalized_entropy = semantic_entropy / max_entropy if max_entropy > 0 else 0
    nsn_score = 1 - normalized_entropy  # Convert to confidence score

    # Step 6: Calculate additional metrics
    fragmentation_ratio = len(clusters) / len(responses)
    largest_cluster_size = max(len(cluster) for cluster in clusters)
    cluster_dominance = largest_cluster_size / len(responses)

    # Step 7: Determine confidence level
    if nsn_score > 0.8 and fragmentation_ratio < 0.4:
        confidence = "🟢 HIGH"
        interpretation = "Responses cluster tightly - coherent understanding"
    elif nsn_score > 0.6 and fragmentation_ratio < 0.6:
        confidence = "🟡 MEDIUM"
        interpretation = "Moderate clustering - some uncertainty present"
    else:
        confidence = "🔴 LOW"
        interpretation = "Highly fragmented responses - model is uncertain"

    # Display results
    print(f"├─ METRICS:")
    print(f"│   ├─ Semantic Entropy:     {semantic_entropy:.3f}")
    print(f"│   ├─ NSN Score:            {nsn_score:.3f}")
    print(f"│   ├─ Fragmentation Ratio:  {fragmentation_ratio:.3f}")
    print(f"│   └─ Cluster Dominance:    {cluster_dominance:.3f}")
    print(f"└─ CONFIDENCE: {confidence}")
    print(f"    └─ {interpretation}")

    return {
        "method": "Normalized Semantic Negentropy",
        "num_responses": n_responses,
        "num_clusters": len(clusters),
        "nsn_score": nsn_score,
        "semantic_entropy": semantic_entropy,
        "fragmentation_ratio": fragmentation_ratio,
        "cluster_dominance": cluster_dominance,
        "clusters": clusters,
        "confidence": confidence,
        "interpretation": interpretation,
        "responses": responses,
    }


# Don't worry about the red warnings that usually get generated here:
print("✅ Set up semantic_entropy_detection")

### Test Semantic Entropy Detection

Let's test this method too with the same question to see how it compares:

> ℹ️ Note that you *could* re-use the `semsmim_bedrock_responses` from earlier directly, but we've chosen to generate new responses to keep things a bit more interactive.

In [ ]:
nsn_bedrock_responses = generate_responses(
    claude_models, "What album was Dua Lipa's 'Hallucinate' featured on?"
)

semantic_entropy_detection(nsn_bedrock_responses)

### Optional: Test with SageMaker Endpoint

Again if you've deployed the SageMaker endpoint(s) in lab 0, you can test them here. Otherwise, skip to the next section:

In [ ]:
nsn_sm_responses = generate_responses(
    gpt_oss_models,
    "What album was Dua Lipa's 'Hallucinate' featured on?",
)

semantic_entropy_detection(nsn_sm_responses)

## 5. Comparative Analysis Across Models
(<a href="#top">Go to top</a>)

Now we've seen a few individual examples, let's compare the results across different detection methods, models, and example questions.

Run the cell below to analyze a small batch of questions with all 3 detection methods, across both our models Claude and GPT-OSS.

⏰ This may take a couple of minutes to complete, and will output the full detailed logs for each combination. Carry on to the following cell to see the overall summary table.

In [ ]:
%%time

test_questions = [
    # Should be pretty straightforward:
    "What's the square root of 16? Reply only with a numeric answer.",
    # Mixed performance with these models:
    "What album was Dua Lipa's 'Hallucinate' featured on?",
    # Future knowledge these models are unlikely to know:
    "What date does AWS re:Invent 2025 start?",
    # *Should* be highly random in response - so low confidence:
    "Think of a random animal. What is it?",
]
model_groups = {"claude": claude_models, "gpt_oss": gpt_oss_models}
methods = {
    "SSD": semantic_similarity_detection,
    "NCP": non_contradiction_detection,
    "NSN": semantic_entropy_detection,
}


def compare_models_and_metrics(
    questions: list[str],
    model_groups: Dict[str, list],
    methods: Dict[str, Callable],
) -> dict:
    """Calculate confidence across a range of questions, models, and metrics at once"""
    results_flat = []
    for q in questions:
        for model_name, models in model_groups.items():
            try:
                responses = generate_responses(models, q)
            except Exception as emodel:
                logger.error("Failed get responses from %s: %s", model_name, emodel)
                for method_name in methods.keys():
                    results_flat.append(
                        {
                            "question": q,
                            "model": model_name,
                            "method": method_name,
                            "confidence": "⚠️ MODEL ERR",
                        }
                    )
                continue
            for method_name, method in methods.items():
                try:
                    result = method(responses)
                    confidence = result["confidence"]
                except Exception as escore:
                    logger.error("Failed to calculate %s: %s", method_name, escore)
                    confidence = "⚠️ SCORE ERR"
                results_flat.append(
                    {
                        "question": q,
                        "model": model_name,
                        "method": method_name,
                        "confidence": confidence,
                    }
                )
    results_df = pd.DataFrame(results_flat)
    return results_df


comparison_df = compare_models_and_metrics(test_questions, model_groups, methods)

Once the analyses are complete, we can view the summary results as a table as below:

In [ ]:
comparison_df.pivot(index="question", columns=["model", "method"]).style.set_properties(
    **{"text-align": "left"}
)

> **Note:** If you see '⚠️ SCORE ERR' or '⚠️ MODEL ERR' entries in your table, you might've run in to throttling errors in our workshop environment. Consider reducing the number of questions, and/or waiting a little before trying again.

What do you see in your results?
- Did both models and all the detection methods consistently score "HIGH" confidence for the "easy" question(s) and "LOW" confidence for the purposefully random ones?
- In areas where the different detection *methods* gave a different score, which one would you agree with based on the detailed logs for that question+model above?
- When the different *models* are scored differently, did this correlate well to one model giving more uncertain or varied answers?

A screenshot of an example output is shown below - although yours might differ a little due to randomness in the generations:

![](img/lab2-sample-comparison-table.png "Screenshot of a pivot table. The questions asked run down the left. The model (claude vs gpt_oss) and detection methods (SSD, NCP, NSN) are nested labels along the top. Each point lists a confidence score (HIGH, MEDIUM, or LOW). The 'Think of a random animal' question scored LOW confidence across all models and detection methods, while the 'square root of 16' question was consistently HIGH. Other questions showed more variable results.")

## 6. Building Production-Ready Agents with Semantic Similarity
(<a href="#top">Go to top</a>)

So far we've applied these techniques on basic LLM calls. But how can they fit in more complex AI applications like agents? In this section, we'll integrate real-time, response-level hallucination detection into an example agent using the Strands Agents SDK.

### What are Strands Agents?

[Strands](https://github.com/strands-agents/sdk-python) is an open-source Python framework for building agentic AI applications. It provides:
- **Unified Model Interface**: Work with multiple LLM providers (including Bedrock, SageMaker, and others outside AWS)
- **Tool Integration**: Connect agents to external tools and APIs
- **Observability**: Built-in tracing and monitoring via OpenTelemetry
- **Extensibility**: Easy to add custom behaviors like hallucination detection

### Our Approach: Semantic Similarity as a Model-Level Guardrail

We'll build an agent that:
1. **Generates multiple candidate responses** every time it calls its core LLM
2. **Analyzes semantic similarity** between the candidate responses
3. **Logs the analysis results** to standard tracing & observability tools
4. **Intervenes if inconsistency is detected** (potential hallucination)
5. **Returns the most reliable response** or raises an alert

#### Why This Matters:

Agents; which typically go through multiple rounds of LLM inference and external tool calls before returning a final answer to the user; are a more complex environment for implementing guardrails than a single LLM call.

Some tool calls, like internal APIs, may be **irreversible, or at least complicated to undo**: So implementing this guardrail at the level of each LLM call (as we've done here) is more generalizable than trying to invoke the *whole agent* several times in parallel and analysing the final responses. However, that approach could also be viable for agents that **only** have information-fetching, read-only tools.

### Step 1: Prepare environment

In [ ]:
# Python Built-Ins:
import base64
import os

# External Dependencies:
from dotenv import load_dotenv
from strands import Agent
from strands.models.bedrock import BedrockModel
from strands.telemetry import StrandsTelemetry

# For our custom parallel model implementation:
import sys

sys.path.append("..")
from hallucination_utils.strands_models.parallel import (
    ParallelModelWithConsolidation,
    ModelResponse,
    TraceAttributes,
)

print("✓ Imports successful!")
print(f"✓ Strands Agent module: {Agent.__module__}")

### Optional: Configure Langfuse for Observability

[Langfuse](https://langfuse.com/) provides production-grade observability for LLM applications. It integrates seamlessly with Strands via OpenTelemetry to track:
- Individual LLM calls and their responses
- Guardrail interventions and scores
- Token usage and latency metrics
- End-to-end conversation traces

The Langfuse configuration will be handled automatically by the `set_up_notebook_langfuse()` function below.

**Note**: You should have already obtained your Langfuse credentials following the **Enable Langfuse tracing** instruction in the Getting Started section. If you haven't set up Langfuse yet, you'll be prompted for your credentials when you run the cell below.

If you prefer not to use Langfuse, you can skip this setup. The agent will work fine without it - you just won't see traces in the Langfuse dashboard.

In [ ]:
# Import tracing utilities
from hallucination_utils.tracing import set_up_notebook_langfuse

# Set up Langfuse (will prompt for credentials if not already configured)
set_up_notebook_langfuse()

### Step 2: Create Base Model

Strands provides integrations to a [wide range](https://strandsagents.com/latest/documentation/docs/api-reference/models/) of Foundation Model providers, and we'll implement the guardrail *on top of* this abstraction so it works for any model Strands supports.

In this case, we'll be wrapping over the same Claude model on Amazon Bedrock:

In [ ]:
# Initialize the base Bedrock model
base_model = BedrockModel(model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0")

print("✓ Base model initialized:", base_model.get_config()["model_id"])

### Step 3: Build Semantic Similarity Model Wrapper

Now we'll set up a custom model wrapper that:
1. **Calls the base model multiple times in parallel** (3x in this example)
2. **Computes semantic similarity** between all candidate responses
3. **Consolidates to the best response** or raises an alert if inconsistency is detected

Most of the complexity is handled by the reusable `ParallelModelWithConsolidation` utility we've provided in [hallucination_utils/strands_models/parallel.py](hallucination_utils/strands_models/parallel.py), which handles:
- Parallel execution of multiple model calls
- Stream processing and response collection (since Strands is stream-based)
- Telemetry integration for observability

Here in the notebook just need to implement the `consolidate()` method with our semantic similarity logic.

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


class SemanticSimilarityModel(ParallelModelWithConsolidation):
    """
    Custom model wrapper that detects hallucinations via semantic similarity analysis.

    This model:
    - Generates N candidate responses in parallel
    - Computes pairwise semantic similarity between all candidates
    - Intervenes if mean similarity falls below threshold
    - Returns the first response if all candidates are consistent
    """

    def __init__(
        self, base_model, num_candidates: int = 3, similarity_threshold: float = 0.75
    ):
        """
        Args:
            base_model: The Strands model to wrap (e.g., BedrockModel)
            num_candidates: Number of parallel responses to generate
            similarity_threshold: Minimum mean similarity score (0-1) to pass
        """
        # Create list of models (repeat the same model N times)
        super().__init__([base_model] * num_candidates)
        self.similarity_threshold = similarity_threshold
        self.encoder = SentenceTransformer("all-MiniLM-L6-v2")

    def consolidate(
        self, alternatives: list[ModelResponse]
    ) -> ModelResponse | tuple[ModelResponse, TraceAttributes]:
        """
        Analyze semantic similarity between candidate responses.

        Returns:
            Consolidated response with telemetry attributes
        """
        # Extract response texts from alternatives
        response_texts = [
            alt.message["content"][0]["text"]
            for alt in alternatives
            if "content" in alt.message and len(alt.message["content"]) > 0
        ]

        if len(response_texts) < 2:
            # Not enough responses to compare - return first one
            return (
                ModelResponse.from_alternatives(
                    alternatives=alternatives, final_message=alternatives[0].message
                ),
                {
                    "gen_ai.guardrail.semantic_similarity.intervened": False,
                    "gen_ai.guardrail.semantic_similarity.status": "insufficient_samples",
                },
            )

        # Step 1: Encode all responses to embeddings
        embeddings = self.encoder.encode(response_texts)

        # Step 2: Calculate pairwise similarities
        similarity_matrix = cosine_similarity(embeddings)

        # Step 3: Get upper triangle (excluding diagonal)
        mask = np.triu(np.ones_like(similarity_matrix, dtype=bool), k=1)
        similarities = similarity_matrix[mask]

        # Step 4: Calculate metrics
        mean_similarity = float(np.mean(similarities))
        min_similarity = float(np.min(similarities))
        max_similarity = float(np.max(similarities))
        std_similarity = float(np.std(similarities))

        # Step 5: Determine if we should intervene
        intervened = mean_similarity < self.similarity_threshold

        # Step 6: Prepare telemetry attributes
        trace_attrs = {
            "gen_ai.guardrail.semantic_similarity.intervened": intervened,
            "gen_ai.guardrail.semantic_similarity.mean_score": mean_similarity,
            "gen_ai.guardrail.semantic_similarity.min_score": min_similarity,
            "gen_ai.guardrail.semantic_similarity.max_score": max_similarity,
            "gen_ai.guardrail.semantic_similarity.std_score": std_similarity,
            "gen_ai.guardrail.semantic_similarity.threshold": self.similarity_threshold,
            "gen_ai.guardrail.semantic_similarity.num_candidates": len(alternatives),
        }

        if intervened:
            # Hallucination detected - you could raise an error, return a warning, etc.
            trace_attrs["gen_ai.guardrail.semantic_similarity.status"] = (
                "INCONSISTENT_RESPONSES"
            )

            print(
                f"\n⚠️ Semantic Similarity Guardrail: Inconsistent responses detected!"
            )
            print(
                "└── Mean similarity: {:.3f} (threshold: {})\n".format(
                    mean_similarity, self.similarity_threshold
                )
            )
        else:
            trace_attrs["gen_ai.guardrail.semantic_similarity.status"] = (
                "CONSISTENT_RESPONSES"
            )
            print(f"\n✓ Semantic Similarity Guardrail: Responses are consistent")
            print(f"└── Mean similarity: {mean_similarity:.3f}\n")

        # Return first response as the consolidated output
        # (In production, you might choose the response with highest avg similarity to others)
        return (
            ModelResponse.from_alternatives(
                alternatives=alternatives, final_message=alternatives[0].message
            ),
            trace_attrs,
        )


print("✓ SemanticSimilarityModel class defined")

### Step 4: Instantiate the Semantic Similarity Model

Now let's create an instance of our guardrail-enabled model.

In [ ]:
# Create semantic similarity model with 3 parallel candidates
semsim_model = SemanticSimilarityModel(
    base_model=base_model,
    num_candidates=3,
    similarity_threshold=0.75,  # Adjust based on your requirements
)

print("✓ Semantic Similarity Model created")

### Step 5: Create and test the Strands Agent

Finally, let's create a Strands Agent that uses our semantic similarity model: So every underlying LLM call is repeated multiple times and checked for consistent outputs.

Note that Strands Agents **remember** interactions (in-memory) by default, so you should re-create the agent when wanting to reset your conversation session.

In [ ]:
# Create agent with our custom model
agent = Agent(model=semsim_model)

In [ ]:
# Ask a simple factual question to start with:
input_message = "What is the capital of France?"
print(f"😊 === User Message: ===\n{input_message}\n========================")
result = agent(input_message)

For testing scenarios with a fresh session each time, we can wrap the agent create and invoke together in a function as shown below:

In [ ]:
def create_and_invoke_semsim_agent(prompt: str):
    agent = Agent(model=semsim_model)
    print(f"😊 === User Message: ===\n{input_message}\n========================")
    result = agent(prompt)
    return result


result = create_and_invoke_semsim_agent(
    "What are the main causes of climate change?",
)

If you enabled Langfuse tracing above, you can also explore the generated traces in the Langfuse UI to see more details.

As shown in the screenshot below, the Langfuse trace should include an extra guardrail 'span' for each place the semantic similarity guardrail was invoked during response generation. The guardrail span should include details like the multiple draft model responses, and the output metrics from the check:

![](./img/langfuse-lab2.png "Screenshot of Langfuse UI showing a trace with a 'semantic_similarity' span indicating where the custom guardrail was called.")

### Production Deployment Considerations

When deploying this pattern to production, consider:

#### 1. Performance vs. Accuracy Tradeoffs
- **Latency**: Running 3 parallel model calls increases latency slightly, but reduces wall-clock time vs. sequential calls
- **Cost**: 3x model invocations = 3x cost per query
- **Optimization**: Use caching, reduce `num_candidates` for non-critical queries, or use faster/cheaper models

#### 2. Threshold Tuning
- **similarity_threshold=0.75** is a starting point - tune based on your domain
- Lower threshold (e.g., 0.6) = fewer false positives, but might miss hallucinations
- Higher threshold (e.g., 0.9) = stricter checking, but might flag legitimate variation
- Use A/B testing and human evaluation to find optimal threshold

#### 3. Error Handling
- Current implementation prints warnings but returns a response anyway
- In production, you might:
  - Raise an exception to force human review
  - Return a confidence score to the user
  - Fallback to a RAG system or web search
  - Log to monitoring system for offline analysis

#### 4. Observability
- Enable Langfuse or another observability platform
- Track metrics: `mean_similarity`, `intervention_rate`, `user_satisfaction`
- Set up alerts when intervention rate exceeds normal baselines
- Review flagged responses to improve prompts and thresholds

#### 5. Advanced Consolidation Strategies
- Current implementation returns the first response
- Alternatives:
  - Return the response with **highest average similarity** to others
  - Use **majority voting** for classification tasks
  - **Blend responses** for creative tasks
  - Apply **secondary checks** (e.g., NLI contradiction detection)

#### 6. Multi-Model Ensembles
- Instead of calling the same model 3x, call different models:
  ```python
  models = [
      BedrockModel("claude-haiku-4-5"),
      BedrockModel("claude-3-5-sonnet"),
      # Could even mix SageMaker models, etc.
  ]
  ensemble_model = SemanticSimilarityModel(models)
  ```
- Provides diversity and can catch model-specific hallucinations

#### 7. Integration with Other Guardrails
- Combine with **Bedrock Guardrails** for content filtering
- Add **contextual grounding** checks if using RAG
- Layer with **Token Probability Level methods** (perplexity, entropy) for more robust detection

<br/>

> **Key Considerations**
>
> These methods have higher latency and computational cost due to multiple model generations and additional processing. However, Section 6 demonstrates how to mitigate this through parallel execution and production optimization strategies. These methods are valuable for high-stakes applications where accuracy is critical, and can be combined with faster methods for different use cases.

## 7. Conclusion and Best Practices
(<a href="#top">Go to top</a>)

### What We've Learned

In this notebook, we've explored three powerful Response Level techniques for detecting LLM hallucinations:

1. **Semantic Similarity Detection** - Fast and effective for detecting response consistency
2. **Non-Contradiction Probability** - Excellent for identifying logical inconsistencies  
3. **Normalized Semantic Negentropy** - Most sophisticated, measuring semantic uncertainty
<br>

| Method | What It Detects | When to Use (High-Level Context) | Why It Works |
|--------|----------------|----------------------------------|--------------|
| **Semantic Similarity Detection** | Inconsistent knowledge across multiple generations | • **Content verification** where consistency matters<br>• **Knowledge validation** with available compute budget<br>• **Quick consistency checks** for important claims | True knowledge should be expressed the same way repeatedly - if answers vary wildly, the model isn't really "sure" |
| **Non-Contradiction Probability** | Logical inconsistencies between responses | • **Factual content** where contradictions are unacceptable<br>• **Research applications** requiring logical coherence<br>• **Decision support** systems with clear right/wrong answers | Real facts don't contradict themselves - if multiple responses conflict, at least one must be wrong |
| **Normalized Semantic Negentropy** | Semantic uncertainty through response clustering | • **High-precision applications** requiring confidence scores<br>• **Research environments** with computational resources<br>• **Advanced quality control** for critical outputs | Confident knowledge produces similar responses that cluster together - scattered responses indicate uncertainty |

### Key Takeaways

- **No single method is perfect** - Different techniques catch different types of hallucinations
- **Model quality varies significantly** - Some models are much more consistent than others
- **Question type matters** - Factual questions and creative prompts require different approaches
- **Confidence thresholds are domain-specific** - Adjust based on your use case criticality

### Best Practices for Production

1. **Use Multiple Methods**: Combine techniques for more robust detection
2. **Set Appropriate Thresholds**: Adjust confidence levels based on domain requirements
3. **Consider Computational Cost**: Balance accuracy with response time needs
4. **Monitor and Adapt**: Continuously evaluate and update detection parameters
5. **Human-in-the-Loop**: Always have escalation paths for uncertain cases

### Next Steps

- **Integrate with existing systems**: Add hallucination detection to your AI pipelines
- **Build domain-specific datasets**: Create evaluation sets for your specific use cases
- **Experiment with ensemble methods**: Combine multiple models and detection techniques
- **Explore fine-tuning**: Train models to be more consistent in your domain

### Additional Resources

- [AWS Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [Sentence Transformers Documentation](https://www.sbert.net/)
- [HuggingFace Transformers Guide](https://huggingface.co/docs/transformers)

---

**Remember**: The goal isn't to eliminate all uncertainty, but to **quantify and manage it** appropriately for your specific use cases.

Happy detecting!

## 8. (Optional) Challenge Exercises
(<a href="#top">Go to top</a>)

<div style="border: 4px solid coral; text-align: left; margin: auto; padding: 20px;">
    <h3>🎯 Try These Exercises on your own time!</h3>
    <p><b>Complete these challenges to deepen your understanding of hallucination detection:</b></p>
    <h4>Beginner Challenges:</h4>
    <ol>
        <li><b>Different Question Types:</b> Test the methods with various question types:
            <ul>
                <li>Historical facts: "When did World War II end?"</li>
                <li>Scientific concepts: "What is photosynthesis?"</li>
                <li>Fictional scenarios: "What is Sherlock Holmes' favorite tea?"</li>
            </ul>
        </li>
        <li><b>Model Comparison:</b> Compare all available models on the same question</li>
        <li><b>Parameter Tuning:</b> Experiment with different temperature values and sample sizes</li>
    </ol>
    <h4>Intermediate Challenges:</h4>
    <ol start="4">
        <li><b>Custom Thresholds:</b> Modify the confidence thresholds in each method</li>
        <li><b>Ambiguous Questions:</b> Test with deliberately ambiguous prompts</li>
        <li><b>Method Combination:</b> Create a combined score using all three methods</li>
    </ol>
    <h4>Advanced Challenges:</h4>
    <ol start="7">
        <li><b>Domain-Specific Testing:</b> Test on medical, legal, or technical questions</li>
        <li><b>Multi-Language Support:</b> Adapt methods for non-English responses</li>
        <li><b>Real-Time Application:</b> Build a web interface for live hallucination detection</li>
    </ol>
</div>